In [2]:
#importing dataset via API call
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")  
project = rf.workspace("so-d4hcz").project("all_finalize")
version = project.version(3)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.64, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to all_finalize--3 in yolov8:: 100%|██████████| 12566/12566 [00:01<00:00, 6905.72it/s]


In [3]:
#importing necessary libraries
from ultralytics import YOLO
import os
import torch
import numpy as np

In [4]:
# Setting up dataset paths
train_images_path = '/content/all_finalize--3/train/images'
train_labels_path = '/content/all_finalize--3/train/labels'
val_images_path = '/content/all_finalize--3/valid/images'
val_labels_path = '/content/all_finalize--3/valid/labels'
test_images_path = '/content/all_finalize--3/test/images'
test_labels_path = '/content/all_finalize--3/test/labels'

In [5]:
#counting no. of classes present 
def count_classes(label_dir):
    class_ids = set()

    #walking through all files in the directory
    for root, _, files in os.walk(label_dir):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                labels = [line.strip().split()[0] for line in f]  
                class_ids.update(map(int, labels))  

    return len(class_ids)


train_labels_path = '/content/all_finalize--3/train/labels'
val_labels_path = '/content/all_finalize--3/valid/labels'
test_labels_path = '/content/all_finalize--3/test/labels'

#count classes
train_classes = count_classes(train_labels_path)
val_classes = count_classes(val_labels_path)
test_classes = count_classes(test_labels_path)

print(f"Number of classes in training set: {train_classes}")
print(f"Number of classes in validation set: {val_classes}")
print(f"Number of classes in test set: {test_classes}")


Number of classes in training set: 27
Number of classes in validation set: 26
Number of classes in test set: 25


In [6]:
#setting actual number of classes
def check_and_fix_labels(label_dir, num_classes):
    for root, _, files in os.walk(label_dir):
        for file in files:
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
                labels = [line.strip().split() for line in lines]
                new_labels = []

                for label in labels:
                    class_id = int(label[0])
                    if class_id >= num_classes:
                        print(f"Fixing {file_path}: Class ID {class_id} exceeds number of classes {num_classes}.")
                        label[0] = str(num_classes - 1)  
                    new_labels.append(' '.join(label))

            with open(file_path, 'w') as f:
                f.write('\n'.join(new_labels))


num_classes = 6

#Fixing labels in each dataset
check_and_fix_labels('/content/all_finalize--3/train/labels', num_classes)
check_and_fix_labels('/content/all_finalize--3/valid/labels', num_classes)
check_and_fix_labels('/content/all_finalize--3/test/labels', num_classes)


Streaming output truncated to the last 5000 lines.
Fixing /content/all_finalize--3/train/labels/Toilet-447-_jpg.rf.5d325778a98687f77f623158c9664565.txt: Class ID 21 exceeds number of classes 6.
Fixing /content/all_finalize--3/train/labels/Toilet-447-_jpg.rf.5d325778a98687f77f623158c9664565.txt: Class ID 22 exceeds number of classes 6.
Fixing /content/all_finalize--3/train/labels/Toilet-447-_jpg.rf.5d325778a98687f77f623158c9664565.txt: Class ID 21 exceeds number of classes 6.
Fixing /content/all_finalize--3/train/labels/Toilet-447-_jpg.rf.5d325778a98687f77f623158c9664565.txt: Class ID 24 exceeds number of classes 6.
Fixing /content/all_finalize--3/train/labels/Kitchen-281-_jpg.rf.6fc7209c69da9e0f5444d7e951501621.txt: Class ID 18 exceeds number of classes 6.
Fixing /content/all_finalize--3/train/labels/Kitchen-281-_jpg.rf.6fc7209c69da9e0f5444d7e951501621.txt: Class ID 11 exceeds number of classes 6.
Fixing /content/all_finalize--3/train/labels/Kitchen-281-_jpg.rf.6fc7209c69da9e0f5444d7e9

In [7]:
# Creating the data.yaml file
data_yaml = """
train: /content/all_finalize--3/train
val: /content/all_finalize--3/valid

nc: 6
names: ['AC','Bathtub','Bed','Bed Frame','Closet','Cupboard']
"""
with open('data.yaml', 'w') as f:
    f.write(data_yaml)

In [8]:
#Loading the pretrained model
model = YOLO('yolov8s.pt')  

100%|██████████| 21.5M/21.5M [00:00<00:00, 252MB/s]


In [9]:
#training the model
model.train(data='data.yaml', epochs=5, imgsz=228, batch=64, device='0')

Ultralytics YOLOv8.2.64 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=data.yaml, epochs=5, time=None, patience=100, batch=64, imgsz=228, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_widt

100%|██████████| 755k/755k [00:00<00:00, 26.0MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 118MB/s]


AMP: checks passed ✅
WARNING ⚠️ imgsz=[228] must be multiple of max stride 32, updating to [256]


...

                   all        520       1325      0.496      0.581      0.529      0.398



5 epochs completed in 0.098 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.64 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


                   all        520       1325      0.492      0.582      0.529      0.398
                    AC         49         53      0.269      0.472       0.31      0.215
               Bathtub         46         46      0.525      0.913      0.787      0.669
                   Bed         16         16      0.389      0.625      0.604      0.423
             Bed Frame          7          7          1      0.426      0.627      0.488
                Closet         15         16      0.284      0.312      0.213      0.168
              Cupboard        304       1187      0.484      0.744      0.635      0.428
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f31baa05120>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [10]:
#evaluating the model
metrics = model.val(data='data.yaml')
print("Validation Metrics: ", metrics)

Ultralytics YOLOv8.2.64 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,127,906 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/all_finalize--3/valid/labels.cache... 520 images, 178 backgrounds, 0 corrupt: 100%|██████████| 520/520 [00:00<?, ?it/s]
os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:10<00:00,  1.20s/it]


                   all        520       1325      0.495      0.581       0.53      0.398
                    AC         49         53       0.27      0.472       0.31      0.212
               Bathtub         46         46      0.532      0.913      0.787      0.669
                   Bed         16         16      0.401      0.625      0.604      0.423
             Bed Frame          7          7          1      0.424      0.627      0.488
                Closet         15         16      0.285      0.312      0.213      0.168
              Cupboard        304       1187      0.484       0.74      0.636      0.428
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 3.4ms postprocess per image
Results saved to runs/detect/train2
Validation Metrics:  ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f31bb327d00>
cu

In [11]:
#testing the model
result = model.predict(source=test_images_path, save=True)
print("Test results saved to: ", result)

Streaming output truncated to the last 5000 lines.
       [[112, 115, 119],
        [112, 115, 119],
        [114, 117, 121],
        ...,
        [ 58,  68,  78],
        [ 57,  67,  77],
        [ 56,  66,  76]],

       [[114, 117, 121],
        [115, 118, 122],
        [116, 119, 123],
        ...,
        [ 59,  69,  79],
        [ 58,  68,  78],
        [ 57,  67,  77]]], dtype=uint8)
orig_shape: (640, 640)
path: '/content/all_finalize--3/test/images/img_wm_revamp-2022-08-20T182033-253_jpg.rf.04d569794aa14796edbe9525a34842ee.jpg'
probs: None
save_dir: 'runs/detect/train3'
speed: {'preprocess': 1.1165142059326172, 'inference': 10.597705841064453, 'postprocess': 1.8696784973144531}, ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'AC', 1: 'Bathtub', 2: 'Bed', 3: 'Bed Frame', 4: 'Closet', 5: 'Cupboard'}
obb: None
orig_img: array([[[196, 208, 214],
        [203, 215, 221],
        [202, 

In [12]:
#initializing result directory
results_dir = '/content/yolov8_results'
os.makedirs(results_dir, exist_ok=True)

In [15]:
#saving the results
try:
    results = model.predict(source=test_images_path, save=True, save_dir=results_dir)
    print(f"Test results saved to: {results_dir}")
except RuntimeError as e:
    print(f"Testing failed: {e}")


image 1/264 /content/all_finalize--3/test/images/101_jpg.rf.f56eb5c3d46f16f35e08d8bb3ee352c5.jpg: 256x256 7 Cupboards, 11.3ms
image 2/264 /content/all_finalize--3/test/images/117_jpg.rf.a6e11628a1b9077c09922af40e05544d.jpg: 256x256 8 Cupboards, 10.1ms
image 3/264 /content/all_finalize--3/test/images/118_jpg.rf.c87da66660321b7c768390a275427a3a.jpg: 256x256 4 Cupboards, 13.0ms
image 4/264 /content/all_finalize--3/test/images/119_jpg.rf.44f15b2e18a2f2004284560a6337d692.jpg: 256x256 8 Cupboards, 7.0ms
image 5/264 /content/all_finalize--3/test/images/135_jpg.rf.a62a7bb9864c8bcd7d81cc10386b3159.jpg: 256x256 5 Cupboards, 7.0ms
image 6/264 /content/all_finalize--3/test/images/153_jpg.rf.3ae778f2549df24a7c30eb56fe282136.jpg: 256x256 2 ACs, 7 Cupboards, 7.1ms
image 7/264 /content/all_finalize--3/test/images/169_jpg.rf.becf43ff7aafcb527c85a7c72cdc157d.jpg: 256x256 2 Cupboards, 8.4ms
image 8/264 /content/all_finalize--3/test/images/170_jpg.rf.1915ac95255ca60d79ae097a57f1a0b2.jpg: 256x256 3 Cupboa